In [1]:
from google.colab import drive

drive.mount('/content/gdrive') # 此處需要登入google帳號

Mounted at /content/gdrive


In [2]:
!pip install transformers==4.5.0 datasets accelerate

     |████████████████████████████████| 2.2MB 6.0MB/s 
     |████████████████████████████████| 225kB 21.9MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 3.3MB 23.9MB/s 
     |████████████████████████████████| 901kB 29.5MB/s 
     |████████████████████████████████| 112kB 43.4MB/s 
     |████████████████████████████████| 245kB 41.2MB/s 


In [3]:
import argparse
import logging
import math
import os
import random
from torch.nn import CrossEntropyLoss
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [4]:
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data.dataloader import DataLoader
from tqdm.auto import tqdm
import torch

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    BertConfig,
    BertForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
    AutoModel,
)
# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()
logger.info(accelerator.state)
datasets.utils.logging.set_verbosity_warning()
transformers.utils.logging.set_verbosity_info()

orign_dataset = load_dataset("glue","mrpc")

05/07/2021 09:03:08 - INFO - __main__ -   Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Use FP16 precision: False



Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [5]:
learning_rate=4e-5
BATCH = 64
EPOCH = 6
num_labels= 2
per_device_train_batch_size=64

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tokenizer = XLMRobertaTokenizer.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384")

tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")

05/07/2021 09:03:13 - INFO - filelock -   Lock 140516395991952 acquired on /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a.lock
https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpqflq__wh


storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
creating metadata file for /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
05/07/2021 09:03:13 - INFO - filelock -   Lock 140516395991952 released on /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a.lock
loading configuration file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b1656

storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
05/07/2021 09:03:13 - INFO - filelock -   Lock 140515190541776 released on /root/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


05/07/2021 09:03:14 - INFO - filelock -   Lock 140515144884048 acquired on /root/.cache/huggingface/transformers/1e5909e4dfaa904617797ed35a6105a23daa56cbefca48fef329f772584699fb.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp_qmffv2q


storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/1e5909e4dfaa904617797ed35a6105a23daa56cbefca48fef329f772584699fb.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/1e5909e4dfaa904617797ed35a6105a23daa56cbefca48fef329f772584699fb.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
05/07/2021 09:03:14 - INFO - filelock -   Lock 140515144884048 released on /root/.cache/huggingface/transformers/1e5909e4dfaa904617797ed35a6105a23daa56cbefca48fef329f772584699fb.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
05/07/2021 09:03:14 - INFO - filelock -   Lock 140515143933648 acquired on /root/.cache/huggingface/transformers/29039dfe8c131360348e9f5ebecd464478cec7576c9af532b55ddcf9d4ec8d1e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock
https://huggingface.co/micr

storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/29039dfe8c131360348e9f5ebecd464478cec7576c9af532b55ddcf9d4ec8d1e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
creating metadata file for /root/.cache/huggingface/transformers/29039dfe8c131360348e9f5ebecd464478cec7576c9af532b55ddcf9d4ec8d1e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
05/07/2021 09:03:14 - INFO - filelock -   Lock 140515143933648 released on /root/.cache/huggingface/transformers/29039dfe8c131360348e9f5ebecd464478cec7576c9af532b55ddcf9d4ec8d1e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
lo

In [7]:
tokenizer

PreTrainedTokenizerFast(name_or_path='microsoft/MiniLM-L12-H384-uncased', vocab_size=30522, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [8]:
class MiniLmClass(torch.nn.Module):
    def __init__(self):
        super(MiniLmClass, self).__init__()
        
        self.MiniLM = AutoModel.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384")
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(384,3)
    def forward(self, input_ids, attention_mask,token_type_ids,labels=None):
        loss_fct = CrossEntropyLoss()
        output= self.MiniLM(input_ids, token_type_ids,attention_mask)
        
        output_2 = self.dropout(output.pooler_output)
        logits = self.classifier(output_2)
        if labels is not None:
          loss = loss_fct(
            logits.view(-1,3), labels.view(-1))
          return loss
        
        return logits

model = MiniLmClass()
model.to(device)

05/07/2021 09:03:15 - INFO - filelock -   Lock 140515144585744 acquired on /root/.cache/huggingface/transformers/12a5ad52cb7fc5542e16e354fe6eb487f2f87edac63bf85dc238b1236dbaf24c.870a3f24d12afafb82ae004a0d0bca699532b9c780d87fc2072696e61a35feea.lock
https://huggingface.co/microsoft/Multilingual-MiniLM-L12-H384/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpph202okv


storing https://huggingface.co/microsoft/Multilingual-MiniLM-L12-H384/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/12a5ad52cb7fc5542e16e354fe6eb487f2f87edac63bf85dc238b1236dbaf24c.870a3f24d12afafb82ae004a0d0bca699532b9c780d87fc2072696e61a35feea
creating metadata file for /root/.cache/huggingface/transformers/12a5ad52cb7fc5542e16e354fe6eb487f2f87edac63bf85dc238b1236dbaf24c.870a3f24d12afafb82ae004a0d0bca699532b9c780d87fc2072696e61a35feea
05/07/2021 09:03:15 - INFO - filelock -   Lock 140515144585744 released on /root/.cache/huggingface/transformers/12a5ad52cb7fc5542e16e354fe6eb487f2f87edac63bf85dc238b1236dbaf24c.870a3f24d12afafb82ae004a0d0bca699532b9c780d87fc2072696e61a35feea.lock
loading configuration file https://huggingface.co/microsoft/Multilingual-MiniLM-L12-H384/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/12a5ad52cb7fc5542e16e354fe6eb487f2f87edac63bf85dc238b1236dbaf24c.870a3f24d12afafb82ae004a0d0bca699532b9c780d87fc2072

storing https://huggingface.co/microsoft/Multilingual-MiniLM-L12-H384/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/e1243df19ff0e1975c063fc4c531ba7fdad1ee538e0d94c41a766febfee0c8ab.3d27c3b243133a56a858d62deffdc59141c45422837cf3fde167b873bad92273
creating metadata file for /root/.cache/huggingface/transformers/e1243df19ff0e1975c063fc4c531ba7fdad1ee538e0d94c41a766febfee0c8ab.3d27c3b243133a56a858d62deffdc59141c45422837cf3fde167b873bad92273
05/07/2021 09:03:29 - INFO - filelock -   Lock 140515144493392 released on /root/.cache/huggingface/transformers/e1243df19ff0e1975c063fc4c531ba7fdad1ee538e0d94c41a766febfee0c8ab.3d27c3b243133a56a858d62deffdc59141c45422837cf3fde167b873bad92273.lock
loading weights file https://huggingface.co/microsoft/Multilingual-MiniLM-L12-H384/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/e1243df19ff0e1975c063fc4c531ba7fdad1ee538e0d94c41a766febfee0c8ab.3d27c3b243133a56a858d62deffdc59141c45422837cf

All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at microsoft/Multilingual-MiniLM-L12-H384.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


MiniLmClass(
  (MiniLM): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(250037, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=Tru

In [9]:
model

MiniLmClass(
  (MiniLM): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(250037, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=Tru

In [10]:
orign_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [11]:
label_list = orign_dataset["train"].features["label"].names
num_labels = len(label_list)
label_list

['not_equivalent', 'equivalent']

In [12]:
def preprocess_function(examples):
    # Tokenize the texts
    
    result = tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        truncation=True,
        max_length=128,
        padding="max_length"
        
        )
    result["labels"]=examples["label"]
    
    return result

In [13]:
preprocess_function(orign_dataset["train"][5])

{'input_ids': [101, 6599, 1999, 1996, 2034, 4284, 1997, 1996, 2095, 3333, 2321, 3867, 2013, 1996, 2168, 2558, 1037, 2095, 3041, 1012, 102, 2007, 1996, 9446, 5689, 2058, 5954, 1005, 1055, 2194, 1010, 6599, 1996, 2034, 4284, 1997, 1996, 2095, 3333, 2321, 3867, 2013, 1996, 2168, 2558, 1037, 2095, 3041, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [14]:
dataset = orign_dataset.map(preprocess_function, batched=True, load_from_cache_file=False,remove_columns=["idx","label","sentence1","sentence2"])

In [15]:
for i in range(1):
  print((dataset["train"][i]))

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 1, 'token_type_ids': [0, 0, 0, 0, 0, 

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 1725
    })
})

In [17]:
train_dataloader = DataLoader(dataset["train"],
                collate_fn=default_data_collator,
                batch_size=per_device_train_batch_size,
                shuffle=True)
eval_dataloader = DataLoader(dataset["validation"],
                collate_fn=default_data_collator,
                batch_size=per_device_train_batch_size,
                shuffle=True)

In [18]:
eval_dataloader

In [19]:
# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.01,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

In [20]:
# Prepare everything with our `accelerator`.
accelerator = Accelerator()

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

metric = load_metric('glue', 'mrpc')

In [21]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

In [23]:
total_batch_size = per_device_train_batch_size * accelerator.num_processes
num_update_steps_per_epoch = len(train_dataloader)
max_train_steps = EPOCH * num_update_steps_per_epoch

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(dataset['train'])}")
logger.info(f"  Num Epochs = {EPOCH}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Total optimization steps = {max_train_steps}")
progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0


for epoch in range(EPOCH):
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(batch["input_ids"],batch["token_type_ids"],batch["attention_mask"],batch["labels"])
        # if step%10==0:
        #   print(batch)
        #   print(outputs)
        loss = outputs
        accelerator.backward(loss)
        optimizer.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        completed_steps += 1

    model.eval()
    for step, batch in enumerate(eval_dataloader):
        outputs = model(batch["input_ids"],batch["token_type_ids"],batch["attention_mask"]) 
        predictions = outputs.argmax(dim=-1)
        # if step%10==0:
        #   print(batch["labels"])
        #   print(predictions)
        metric.add_batch(
            predictions=accelerator.gather(predictions),
            references=accelerator.gather(batch["labels"]),
        )

    eval_metric = metric.compute()
    logger.info(f"epoch {epoch}: {eval_metric}")

05/07/2021 09:04:20 - INFO - __main__ -   ***** Running training *****
05/07/2021 09:04:20 - INFO - __main__ -     Num examples = 3668
05/07/2021 09:04:20 - INFO - __main__ -     Num Epochs = 6
05/07/2021 09:04:20 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 64
05/07/2021 09:04:20 - INFO - __main__ -     Total optimization steps = 348


05/07/2021 09:05:19 - INFO - /usr/local/lib/python3.7/dist-packages/datasets/metric.py -   Removing /root/.cache/huggingface/metrics/glue/mrpc/default_experiment-1-0.arrow
05/07/2021 09:05:19 - INFO - __main__ -   epoch 0: {'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}
05/07/2021 09:06:16 - INFO - /usr/local/lib/python3.7/dist-packages/datasets/metric.py -   Removing /root/.cache/huggingface/metrics/glue/mrpc/default_experiment-1-0.arrow
05/07/2021 09:06:16 - INFO - __main__ -   epoch 1: {'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}
05/07/2021 09:07:14 - INFO - /usr/local/lib/python3.7/dist-packages/datasets/metric.py -   Removing /root/.cache/huggingface/metrics/glue/mrpc/default_experiment-1-0.arrow
05/07/2021 09:07:14 - INFO - __main__ -   epoch 2: {'accuracy': 0.7426470588235294, 'f1': 0.8192771084337349}
05/07/2021 09:08:12 - INFO - /usr/local/lib/python3.7/dist-packages/datasets/metric.py -   Removing /root/.cache/huggingface/metrics/glue/mrpc/default_exp

In [24]:
PATH="/content/gdrive/MyDrive/Colab Notebooks/model/minilm/mini.bin"

In [25]:
torch.save(model.state_dict(), PATH)

# inference

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
label_list = ['not_equivalent', 'equivalent']

In [35]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")

loading configuration file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/49c302ee103bf6737d0877

In [38]:
model = MiniLmClass()
model = model.to(device)
model.eval()


loading configuration file https://huggingface.co/microsoft/Multilingual-MiniLM-L12-H384/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/12a5ad52cb7fc5542e16e354fe6eb487f2f87edac63bf85dc238b1236dbaf24c.870a3f24d12afafb82ae004a0d0bca699532b9c780d87fc2072696e61a35feea
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250037
}

loading weights file https://huggingface.co/microsoft/Multilingual-MiniLM-L12-H384/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transfor

MiniLmClass(
  (MiniLM): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(250037, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=Tru

In [40]:
sentence1 = "Earnings per share from recurring operations will be 13 cents to 14 cents ."
sentence2 = "That beat the company 's April earnings forecast of 8 to 9 cents a share ."
tokenized_input = tokenizer(sentence1, sentence2, return_tensors="pt").to(device)
outputs = model(**tokenized_input)
print(f"Prediction: {label_list[outputs.argmax(dim=-1).item()]}")

Prediction: not_equivalent
